# Home assignment
### Skvortsov Ivan, Altunina Anastasiya, Romanenko Alexandra, Grigoryan Mikhail, Peshkov Maksim

*email to pandreyanov@gmail.com, use ipnb, use markdown, explain what you do*

In [26]:
import numpy as np
import pandas as pd
import requests
from pprint import pprint
import gzip
import shutil
from tqdm import tqdm
import sqlite3
from bs4 import BeautifulSoup
import json

from tqdm.notebook import tqdm
from pymongo import MongoClient
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

## Assignment 1 (1 points): download all 7 dumps, print summary

First we define the IMDB url and datasets' file names.

In [2]:
host = 'https://datasets.imdbws.com/'

sources = [
    'name.basics.tsv.gz',
    'title.akas.tsv.gz',
    'title.basics.tsv.gz',
    'title.crew.tsv.gz',
    'title.episode.tsv.gz',
    'title.principals.tsv.gz',
    'title.ratings.tsv.gz'
]

This function downloads `.gz` zipped files (if necessary) and unzips them:

In [3]:
def get_data(host, sources, download = True):
    if download:
        for source in tqdm(sources):
            url = host + source
            r = requests.get(url)
            open(source, 'wb').write(r.content)
        print('Data downloading complete!')

    for source in tqdm(sources):
        with gzip.open(source, 'rb') as f_in:
            with open(source.replace('.gz', ''), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    print('Unzipping complete!')

In [4]:
get_data(host=host, sources=sources)

  0%|          | 0/7 [00:00<?, ?it/s]

Data downloading complete!


  0%|          | 0/7 [00:00<?, ?it/s]

Unzipping complete!


For large datasets, Pandas fails to define columns' types properly, falling for the default (`object`). Since we don't need to have proper dtypes at this point and don't want to inflate memory, we just ignore the `DtypeWarning`.

Code below loads `.tsv` files as Pandas dataframes and returns their infos.

In [5]:
import warnings
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

for datafile in [i.replace('.gz', '') for i in sources]:
    df = pd.read_csv(datafile, sep='\t', na_values=['\\N', ])
    print(f'TABLE: {datafile}')
    display(df.info())
    print('\n')
    del df

warnings.filterwarnings("default", category=pd.errors.DtypeWarning)

TABLE: name.basics.tsv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11235761 entries, 0 to 11235760
Data columns (total 6 columns):
 #   Column             Dtype  
---  ------             -----  
 0   nconst             object 
 1   primaryName        object 
 2   birthYear          float64
 3   deathYear          float64
 4   primaryProfession  object 
 5   knownForTitles     object 
dtypes: float64(2), object(4)
memory usage: 514.3+ MB


None



TABLE: title.akas.tsv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29161555 entries, 0 to 29161554
Data columns (total 8 columns):
 #   Column           Dtype  
---  ------           -----  
 0   titleId          object 
 1   ordering         int64  
 2   title            object 
 3   region           object 
 4   language         object 
 5   types            object 
 6   attributes       object 
 7   isOriginalTitle  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 1.7+ GB


None



TABLE: title.basics.tsv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8279338 entries, 0 to 8279337
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   tconst          object 
 1   titleType       object 
 2   primaryTitle    object 
 3   originalTitle   object 
 4   isAdult         float64
 5   startYear       float64
 6   endYear         float64
 7   runtimeMinutes  object 
 8   genres          object 
dtypes: float64(3), object(6)
memory usage: 568.5+ MB


...

None



TABLE: title.principals.tsv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46864751 entries, 0 to 46864750
Data columns (total 6 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   tconst      object
 1   ordering    int64 
 2   nconst      object
 3   category    object
 4   job         object
 5   characters  object
dtypes: int64(1), object(5)
memory usage: 2.1+ GB


None



TABLE: title.ratings.tsv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1187512 entries, 0 to 1187511
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1187512 non-null  object 
 1   averageRating  1187512 non-null  float64
 2   numVotes       1187512 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 27.2+ MB


None

Just to prove that we really downloaded everything 😅

In [6]:
!ls

log		    title.basics.tsv	  title.principals.tsv
name.basics.tsv     title.basics.tsv.gz   title.principals.tsv.gz
name.basics.tsv.gz  title.crew.tsv	  title.ratings.tsv
sample_data	    title.crew.tsv.gz	  title.ratings.tsv.gz
title.akas.tsv	    title.episode.tsv
title.akas.tsv.gz   title.episode.tsv.gz


## Assignment 2 (2 points): save them into 7 sqlite databases with same column names

Simple function for creating sqlite databases based on a given `.tsv` file. This time we set the `low_memory` argument to False to import our dtypes properly.

In [7]:
def create_db(datafile):
    database = datafile.replace('.tsv', '.db')
    conn = sqlite3.connect(database)
    c = conn.cursor()

    df = pd.read_csv(datafile, sep='\t', low_memory=False, na_values=['\\N', ])

    df.to_sql(
        name=datafile.replace('.tsv', '').upper().replace('.', '_'), 
        con=conn, 
        if_exists='replace', 
        index=False)

    del df # we don't need this im memory anymore

In [8]:
for source in sources:
    datafile = source.replace('.gz', '')
    create_db(datafile=datafile)

In [9]:
!ls

log		    title.basics.db	 title.episode.tsv.gz
name.basics.db	    title.basics.tsv	 title.principals.db
name.basics.tsv     title.basics.tsv.gz  title.principals.tsv
name.basics.tsv.gz  title.crew.db	 title.principals.tsv.gz
sample_data	    title.crew.tsv	 title.ratings.db
title.akas.db	    title.crew.tsv.gz	 title.ratings.tsv
title.akas.tsv	    title.episode.db	 title.ratings.tsv.gz
title.akas.tsv.gz   title.episode.tsv


Let's retrieve some data from one of the databases (`name.basics`).
First option is just by using `pd.read_sql` function:

In [10]:
datafile = sources[0].replace('.gz', '')
database = datafile.replace('.tsv', '.db')
conn = sqlite3.connect(database)
pd.read_sql('select * from NAME_BASICS limit 10', conn)

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0031983,tt0072308"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,soundtrack,music_department","tt0056404,tt0049189,tt0054452,tt0057345"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,soundtrack,writer","tt0080455,tt0078723,tt0072562,tt0077975"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0083922,tt0060827,tt0050986,tt0050976"
5,nm0000006,Ingrid Bergman,1915.0,1982.0,"actress,soundtrack,producer","tt0077711,tt0034583,tt0036855,tt0038109"
6,nm0000007,Humphrey Bogart,1899.0,1957.0,"actor,soundtrack,producer","tt0033870,tt0042593,tt0043265,tt0034583"
7,nm0000008,Marlon Brando,1924.0,2004.0,"actor,soundtrack,director","tt0047296,tt0068646,tt0078788,tt0070849"
8,nm0000009,Richard Burton,1925.0,1984.0,"actor,soundtrack,producer","tt0057877,tt0061184,tt0087803,tt0059749"
9,nm0000010,James Cagney,1899.0,1986.0,"actor,soundtrack,director","tt0042041,tt0035575,tt0029870,tt0031867"


Second option is as was presented on the seminar:

In [11]:
data = []
c = conn.cursor()
c.execute("SELECT * from NAME_BASICS")

for f in c.fetchall():
    data.append(f)

conn.commit()
conn.close()

In [12]:
pd.DataFrame(data[:10], columns = ['nconst', 'primaryName', 'birthYear', 'deathYear', 'primaryProfession', 'knownForTitles'])

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0031983,tt0072308"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack","tt0117057,tt0037382,tt0038355,tt0071877"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,soundtrack,music_department","tt0056404,tt0049189,tt0054452,tt0057345"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,soundtrack,writer","tt0080455,tt0078723,tt0072562,tt0077975"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0083922,tt0060827,tt0050986,tt0050976"
5,nm0000006,Ingrid Bergman,1915.0,1982.0,"actress,soundtrack,producer","tt0077711,tt0034583,tt0036855,tt0038109"
6,nm0000007,Humphrey Bogart,1899.0,1957.0,"actor,soundtrack,producer","tt0033870,tt0042593,tt0043265,tt0034583"
7,nm0000008,Marlon Brando,1924.0,2004.0,"actor,soundtrack,director","tt0047296,tt0068646,tt0078788,tt0070849"
8,nm0000009,Richard Burton,1925.0,1984.0,"actor,soundtrack,producer","tt0057877,tt0061184,tt0087803,tt0059749"
9,nm0000010,James Cagney,1899.0,1986.0,"actor,soundtrack,director","tt0042041,tt0035575,tt0029870,tt0031867"


## Assignment 3 (3 points): download 2 dumps (actors-movie and movie-rating) and produce a pandas dataframe of actor-avgrating using join

At first we need to get needed columns from `name.basics` and change the format of column `knownForTitles` so we can use `df.explode`:

In [13]:
datafile = sources[0].replace('.gz', '')
database = datafile.replace('.tsv', '.db')
conn = sqlite3.connect(database)
conn = sqlite3.connect(database)
data_actors = pd.read_sql('select primaryName, knownForTitles from NAME_BASICS', conn)

In [14]:
data_actors.knownForTitles = data_actors.knownForTitles.str.split(',')

In [15]:
df_new = data_actors.explode(column = 'knownForTitles')

Now we need info from `title.ratings` and we can inner join them by `knownForTitles`

In [16]:
datafile = sources[6].replace('.gz', '')
database = datafile.replace('.tsv', '.db')
conn = sqlite3.connect(database)
data_ratings = pd.read_sql('select tconst, averageRating from TITLE_RATINGS', conn).rename(columns={'tconst': 'knownForTitles'})
#df_final = df_new.set_index('knownForTitles').join(data_ratings.set_index('knownForTitles'), how = 'left')
#df_final = df_final.groupby(by = 'primaryName').mean()

In [17]:
df_final = df_new.set_index('knownForTitles').join(data_ratings.set_index('knownForTitles'), how = 'inner')

In [18]:
df_final = df_final.groupby(by = 'primaryName').mean()

Let's take a look on final table:

In [19]:
df_final.head()

,averageRating
primaryName,
!'aru Ikhuisi Piet Berendse,7.3
!Mystery Girl!,4.4
!Nqate Xqamxebe,7.5
$2 Tony,5.6
$haniqua,8.5


## Assignment 4 (4 points): download 1,000 movies from api into nosql database (mongo or something else)

On this website we found how install mongo in Google Colaboratory https://gist.github.com/korakot/64f4e589eff252cf0486388da26c17fd

In [27]:
!apt install mongodb >log
!service mongodb start

!pip install IMDbPY
import imdb
from tqdm.notebook import tqdm
from pymongo import MongoClient
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ia = imdb.IMDb()



 * Starting database mongodb
   ...done.


In [30]:
from pymongo import MongoClient

client = MongoClient()
db = client['movies_imdb']
collection1 = db['films_details']

Function to collect some info about films

In [31]:
def need_details(mov): 
  one_movie = {}
  for det in ['title', 'kind', 'genres', 'countries','year','imdbID']:
    try:
      one_movie[det] = mov[det]
    except:
      one_movie[det] = []
  return one_movie

Big 'while loop' to collect all data about films using function

In [32]:
i = 0 #how many films are found in api
j = 1 #number to find in api

while i < 1000 and j < 100000: #we hope that in 100000 attempts we can find 1000 good films
  if j < 10:
    try:
      movie = ia.get_movie('000000'+str(j)).data
      if len(movie) == 0:
          pass
      else:
        i += 1
        collection1.insert_one(need_details(movie))
    except:
      pass
  
  if (j < 100) and (j >= 10):
    try:
      movie = ia.get_movie('00000'+str(j)).data
      if len(movie) == 0:
          pass
      else:
        i += 1
        collection1.insert_one(need_details(movie))
    except:
      pass
  
  if (j < 1000) and (j >= 100):
    try:
      movie = ia.get_movie('0000'+str(j)).data
      if len(movie) == 0:
          pass
      else:
        i += 1
        collection1.insert_one(need_details(movie))
    except:
      pass

  if (j >= 1000) and (j<10000):
    try:
      movie = ia.get_movie('000'+str(j)).data
      if len(movie) == 0:
          pass
      else:
        i += 1
        collection1.insert_one(need_details(movie))
    except:
      pass
      
  if (j >= 10000):
      try:
        movie = ia.get_movie('00'+str(j)).data
        if len(movie) == 0:
            pass
        else:
          i += 1
          collection1.insert_one(need_details(movie))
      except:
        pass

  
  if i % 20 == 0: #to display how long it works
    %%time
    print(i,j)
    clear_output(wait=True)

  j += 1

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
1000 1001


Example of info:

In [33]:
result = client.movies_imdb.films_details.find()
next(result)

{'_id': ObjectId('614929416dd33649b0f6cd27'),
 'countries': ['United States'],
 'genres': ['Documentary', 'Short'],
 'imdbID': '0000001',
 'kind': 'movie',
 'title': 'Carmencita',
 'year': 1894}

In [34]:
result.count()

1000

First 4 films

In [15]:
#for doc in result:
#  print(doc)

{'_id': ObjectId('6149284c6dd33649b0f6cd0a'), 'title': 'Le clown et ses chiens', 'kind': 'movie', 'genres': ['Animation', 'Short'], 'countries': ['France'], 'year': 1892, 'imdbID': '0000002'}
{'_id': ObjectId('6149284d6dd33649b0f6cd0b'), 'title': 'Poor Pierrot', 'kind': 'movie', 'genres': ['Animation', 'Comedy', 'Short', 'Romance'], 'countries': ['France'], 'year': 1892, 'imdbID': '0000003'}
{'_id': ObjectId('6149284f6dd33649b0f6cd0c'), 'title': 'Un bon bock', 'kind': 'movie', 'genres': ['Animation', 'Short'], 'countries': ['France'], 'year': 1892, 'imdbID': '0000004'}
{'_id': ObjectId('614928506dd33649b0f6cd0d'), 'title': 'Blacksmith Scene', 'kind': 'movie', 'genres': ['Short', 'Comedy'], 'countries': ['United States'], 'year': 1893, 'imdbID': '0000005'}


## Assignment 5 (5 points): parse 1,000 movies from website into nosql database (mongo or something else)

Let's parse IMDb using soup. Take the url, get info and save to DataFrame. At the end we get a DataFrame with 1100 strings and information about name and alternative name of the film,  it;s genre and date it was published, avgRating and name of it's director

In [35]:
def get_film_info(link):
    
    url = f'https://www.imdb.com/title/{link}'
    res = requests.get(url)
    if str(res) == '<Response [200]>':
      soup = BeautifulSoup(res.text)

      myjson = soup.find('script', attrs={"type":"application/ld+json"}).string

      smaller_dict = lambda x, y: dict([(i,x[i]) for i in x if i in set(y)])

      keys = ["url", "name", "alternateName", "genre", 'datePublished', 'aggregateRating', 'director']
      my_dict = smaller_dict(eval(myjson), keys)

      my_dict['director'] = my_dict.get('director', np.nan)
      if type(my_dict['director']) == list:
          my_dict['director'] = my_dict['director'][0]['name']
          
      my_dict['aggregateRating'] = my_dict.get('aggregateRating')
      if type(my_dict['aggregateRating']) == dict:
          my_dict['aggregateRating'] = my_dict['aggregateRating']['ratingValue']
      my_dict['genre'] = my_dict.get('genre', np.nan)
      if type(my_dict['genre']) == list:
          my_dict['genre'] = ', '.join(i for i in my_dict['genre'])
      
      return my_dict

Now we just insert dict with some info about movie into mongodb:



In [38]:
from pymongo import MongoClient

client = MongoClient()
db = client['movies_imdb2']
collection = db['films_details2']

#df = pd.DataFrame(columns=['url', 'name', 'alternateName', 'aggregateRating', 'genre',
#       'datePublished', 'director'])
for i in tqdm(range(1, 1050)):
  if i < 10:
    res = get_film_info(link=f'tt000000{i}/')
  elif i < 100:
    res = get_film_info(link=f'tt00000{i}/')
  elif i < 1000:
    res = get_film_info(link=f'tt0000{i}/')
  else:
    res = get_film_info(link=f'tt000{i}/')
  try:
    collection.insert_one(res)
  except:
    pass
    #df = pd.concat([df, res])

  0%|          | 0/1049 [00:00<?, ?it/s]

Example of movie info:

In [39]:
result2 = client.movies_imdb2.films_details2.find()
next(result2)

{'_id': ObjectId('61492efc6dd33649b0f6d115'),
 'aggregateRating': 5.7,
 'datePublished': '1894-03-10',
 'director': 'William K.L. Dickson',
 'genre': 'Documentary, Short',
 'name': 'Carmencita',
 'url': '/title/tt0000001/'}

Let's see how many items we have in collection:

In [40]:
result2.count()

1048